In [1]:
import numpy as np
from scipy import linalg
from astropy.io import fits

In [2]:
theory = np.genfromtxt('tests/sample_files/theory.txt')
binned = np.genfromtxt('tests/sample_files/bin_ordered_data.dat')
noisey = np.genfromtxt('tests/sample_files/noise_mean.txt')
covariance = np.genfromtxt('tests/sample_files/covariance.txt')
inv_covariance = np.genfromtxt('tests/sample_files/inv_covariance.txt')

In [3]:
print(np.allclose(binned, noisey, rtol = 0.0000001))

False


In [4]:
binned_re = binned.reshape(30,-1)
theory_re = theory.reshape(30,-1)

overall_order = ['xi_plus', 'xi_minus']
bin_order = ['bin_1_1', 
             'bin_2_1', 'bin_2_2', 
             'bin_3_1', 'bin_3_2', 'bin_3_3', 
             'bin_4_1', 'bin_4_2', 'bin_4_3', 'bin_4_4', 
             'bin_5_1', 'bin_5_2', 'bin_5_3', 'bin_5_4', 'bin_5_5']
ordered_bins = []
for order in overall_order:
    for bin in bin_order:
        ordered_bins.append(order + '_' + bin)

new_order = [None] * len(ordered_bins)

In [6]:
for i, proper in enumerate(theory_re):
    for j, binned in enumerate(binned_re):
        if np.allclose(proper, binned) is True:
            new_order[i] = ordered_bins[j]

In [7]:
print(new_order)

['xi_plus_bin_1_1', 'xi_plus_bin_2_1', 'xi_plus_bin_3_1', 'xi_plus_bin_4_1', 'xi_plus_bin_5_1', 'xi_plus_bin_2_2', 'xi_plus_bin_3_2', 'xi_plus_bin_4_2', 'xi_plus_bin_5_2', 'xi_plus_bin_3_3', 'xi_plus_bin_4_3', 'xi_plus_bin_5_3', 'xi_plus_bin_4_4', 'xi_plus_bin_5_4', 'xi_plus_bin_5_5', 'xi_minus_bin_1_1', 'xi_minus_bin_2_1', 'xi_minus_bin_3_1', 'xi_minus_bin_4_1', 'xi_minus_bin_5_1', 'xi_minus_bin_2_2', 'xi_minus_bin_3_2', 'xi_minus_bin_4_2', 'xi_minus_bin_5_2', 'xi_minus_bin_3_3', 'xi_minus_bin_4_3', 'xi_minus_bin_5_3', 'xi_minus_bin_4_4', 'xi_minus_bin_5_4', 'xi_minus_bin_5_5']


In [7]:
covariance_transposed = np.transpose(covariance)
print(np.allclose(covariance_transposed, covariance, rtol = 0.0000001))

True


In [8]:
# Checking to see if the KCAP inverse covariance is symmetric

inv_covariance_transposed = np.transpose(inv_covariance)
print(np.allclose(inv_covariance, inv_covariance_transposed, rtol = 0.0000001))

False


In [9]:
# Checking to see if the numpy method of inverting a covariance produces the same inverse covariance as KCAP

inv_covariance_numpy = np.linalg.inv(covariance)
print(np.allclose(inv_covariance, inv_covariance_numpy, rtol = 0.0000001))

True


In [10]:
# Checking if the determinants are the same

det_cov = np.linalg.det(covariance)
det_inv_cov = np.linalg.det(inv_covariance)
print(np.allclose(det_cov, 1/det_inv_cov, rtol = 0.0000001))

True


/home/ruyi/anaconda3/envs/kcap_env/lib/python3.9/site-packages/numpy/linalg/linalg.py:2159: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [11]:
# Check if scipy inverse is the same as the KCAP inverse

inv_covariance_scipy = linalg.inv(covariance)
print(np.allclose(inv_covariance, inv_covariance_scipy, rtol = 0.0000001))
print(np.allclose(inv_covariance, inv_covariance_scipy, rtol = 0.00000001))

True
False


In [12]:
# Check if the scipy inverse is symmetric

scipy_inv_covariance_transposed = np.transpose(inv_covariance_scipy)
print(np.allclose(scipy_inv_covariance_transposed, inv_covariance_scipy, rtol = 0.0000001))

False


In [13]:
scipy_numpy_diff = inv_covariance_scipy - inv_covariance_numpy

In [14]:
boosted_covariance = np.copy(covariance)
for i, val in enumerate(boosted_covariance):
    # boosted_covariance[i][i] = boosted_covariance[i][i]*2*10**len(val)
    boosted_covariance[i][i] = boosted_covariance[i][i]*10**10

In [15]:
inv_boosted_covariance = linalg.inv(boosted_covariance)
print(np.allclose(np.transpose(inv_boosted_covariance), inv_boosted_covariance, rtol = 0.0000001))

True


In [16]:
unity = np.dot(inv_covariance, covariance)
symmetrised_inv_covariance = (inv_covariance + inv_covariance.T)/2
symmetrised_unity = np.dot(symmetrised_inv_covariance, covariance)
symmetrised_unity_det = np.linalg.det(symmetrised_unity)
unity_det = np.linalg.det(unity)

In [17]:
hdul = fits.open('tests/sample_files/kids_covariance.fits')
hdul.info()
cov_data = hdul[1].data

Filename: tests/sample_files/kids_covariance.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  COVMAT        1 ImageHDU        13   (270, 270)   float64   
  2  xiP           1 BinTableHDU     29   135R x 5C   [K, K, K, D, D]   
  3  xiM           1 BinTableHDU     29   135R x 5C   [K, K, K, D, D]   
  4  NZ_SOURCE     1 BinTableHDU     36   119R x 8C   [D, D, D, D, D, D, D, D]   


In [18]:
print(np.allclose(covariance, cov_data, rtol = 0.0000001))

True


In [19]:
w, v = np.linalg.eig(covariance)
inv_eigenvals = np.zeros(shape = (270,270))

for i, val in enumerate(w):
    if val > 1.*10**(-10):
        inv_eigenvals[i][i] += 1/val
    else:
        pass

inv_eigenvals_diag = np.diag(inv_eigenvals)

eigen_inv_covariance = np.dot(v, np.dot(inv_eigenvals, v.T))
print(np.allclose(eigen_inv_covariance, eigen_inv_covariance.T, 0.0000001))
eigen_unity = np.dot(eigen_inv_covariance, covariance)
eigen_unity_det = np.linalg.det(eigen_unity)
diag = np.diag(eigen_unity)
# inv_eigenvals = 1/w

True


In [20]:
L = np.linalg.cholesky(covariance)
identity = np.identity(270)
y = np.linalg.solve(L, identity)
x = np.linalg.solve(L.T, y)

print(np.allclose(x, x.T, 0.0000001))

True


In [21]:
cholesky_unity = np.dot(x, covariance)
cholestky_unity_det = np.linalg.det(cholesky_unity)

In [22]:
test_identity = np.identity(12)
np.linalg.det(test_identity)

1.0

In [30]:
suppressed_covariance = np.zeros(shape=(270,270))
for i, cov_slice in enumerate(covariance):
    for j, val in enumerate(cov_slice):
        suppressed_covariance[i][j] = val * 0.001 ** (abs(i-j))
inv_new_covariance = np.linalg.inv(suppressed_covariance)

In [31]:
print(np.allclose(suppressed_covariance, suppressed_covariance.T, atol = 0.001))
print(np.allclose(inv_new_covariance, inv_new_covariance.T, atol = 0.001, equal_nan = True))

True
False


In [32]:
print(np.linalg.det(inv_new_covariance))

inf


/home/ruyi/anaconda3/envs/kcap_env/lib/python3.9/site-packages/numpy/linalg/linalg.py:2159: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [37]:
test_matrix = np.array([[1,1,1],[1,1,1],[1,1,1]])
suppressed_matrix = np.zeros(shape=(3,3))
for i, mat_slice in enumerate(test_matrix):
    for j, val in enumerate(mat_slice):
        suppressed_matrix[i][j] += val * 0.9 ** (abs(i-j))

print(suppressed_matrix)

1.0
0.9
0.81
0.9
1.0
0.9
0.81
0.9
1.0
[[1.   0.9  0.81]
 [0.9  1.   0.9 ]
 [0.81 0.9  1.  ]]


# Looking at the kids fits file

In [8]:
# hdul1 = fits.open('/home/ruyi/cosmology/kcap/runs/lfi_config/xipm_KIDS1000_BlindC_with_m_bias_V1.0.0A_ugriZYJHKs_photoz_SG_mask_LF_svn_309c_2Dbins_v2_goldclasses_Flag_SOM_Fid.fits')
hdul = fits.open('/home/ruyi/cosmology/kcap/runs/lfi_config/delfi_kids_fits_noiseless.fits')

In [9]:
hdul.info()

Filename: /home/ruyi/cosmology/kcap/runs/lfi_config/delfi_kids_fits_noiseless.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  COVMAT        1 ImageHDU        13   (270, 270)   float64   
  2  xiP           1 BinTableHDU     29   135R x 5C   [K, K, K, D, D]   
  3  xiM           1 BinTableHDU     29   135R x 5C   [K, K, K, D, D]   
  4  NZ_SOURCE     1 BinTableHDU     36   119R x 8C   [D, D, D, D, D, D, D, D]   


In [12]:
hdul[2].data

FITS_rec([(1, 1, 1, -7.77276009e-05,   0.71336491),
          (1, 1, 2, -3.52416165e-06,   1.45209561),
          (1, 1, 3, -9.50491139e-06,   2.95582478),
          (1, 1, 4,  1.97019197e-05,   6.01675264),
          (1, 1, 5,  2.39543778e-06,  12.24744871),
          (1, 1, 6, -5.35730592e-06,  24.93039168),
          (1, 1, 7,  2.75091742e-06,  50.74725716),
          (1, 1, 8,  9.63998265e-07, 103.29898312),
          (1, 1, 9, -4.44588679e-07, 210.27106705),
          (1, 2, 1,  8.63755274e-05,   0.71336491),
          (1, 2, 2,  1.64981643e-05,   1.45209561),
          (1, 2, 3, -2.40071613e-05,   2.95582478),
          (1, 2, 4, -1.18342989e-06,   6.01675264),
          (1, 2, 5,  2.13667027e-06,  12.24744871),
          (1, 2, 6, -3.13803476e-06,  24.93039168),
          (1, 2, 7,  5.00681187e-07,  50.74725716),
          (1, 2, 8,  8.75093925e-07, 103.29898312),
          (1, 2, 9,  2.33044691e-07, 210.27106705),
          (1, 3, 1, -3.85576686e-05,   0.71336491),
          (1

In [8]:
hdul[2].data

FITS_rec([(1, 1, 1,  1.25115625e-04,   0.71336491),
          (1, 1, 2,  6.00683567e-06,   1.45209561),
          (1, 1, 3,  6.19317502e-05,   2.95582478),
          (1, 1, 4,  2.11742421e-06,   6.01675264),
          (1, 1, 5, -2.92882256e-06,  12.24744871),
          (1, 1, 6, -2.38991012e-06,  24.93039168),
          (1, 1, 7, -1.56889835e-06,  50.74725716),
          (1, 1, 8,  1.61541556e-07, 103.29898312),
          (1, 1, 9, -3.29682598e-09, 210.27106705),
          (1, 2, 1,  5.11868980e-05,   0.71336491),
          (1, 2, 2,  2.82597971e-05,   1.45209561),
          (1, 2, 3,  4.33571094e-06,   2.95582478),
          (1, 2, 4,  9.95191063e-06,   6.01675264),
          (1, 2, 5,  2.48591965e-06,  12.24744871),
          (1, 2, 6,  8.84065085e-07,  24.93039168),
          (1, 2, 7,  1.98120807e-06,  50.74725716),
          (1, 2, 8,  3.05569286e-07, 103.29898312),
          (1, 2, 9,  1.07653468e-07, 210.27106705),
          (1, 3, 1, -3.04513359e-05,   0.71336491),
          (1